In [ ]:
import pandas as pd 
import numpy as np
from  sklearn.compose import ColumnTransformer
from  sklearn.impute import SimpleImputer
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn import preprocessing

In [ ]:
# Saves content of csv file to HealthData variable
HealthData= pd.read_csv('Mental_Health_Care_in_the_Last_4_Weeks.csv')
HealthData

In [ ]:
#  To get all  data type
HealthData.info()

In [ ]:
# To check dataset statistics
HealthData.describe()

In [ ]:
# Check for ? and replaces ? with NaN so 
# the system can see it as missing value
HealthData= HealthData.replace('?', np.nan)
HealthData.isnull().sum()



In [ ]:
HealthData.head()

In [ ]:
HealthData.drop(['Suppression Flag'], axis=1, inplace=True)

In [ ]:
HealthData.drop(['Confidence Interval'], axis=1, inplace=True)

In [ ]:
HealthData.drop(['Subgroup'], axis=1, inplace=True)

In [ ]:

HealthData.drop(['Time Period Label'], axis=1, inplace=True)

In [ ]:
HealthData.shape

Feature Quartile Range has range values that can not be interperted by the system to solve this, Quartile Range is slipt into low and high then dropped.

In [ ]:
lows, highs = [], []

for data in HealthData['Quartile Range']:
    if isinstance(data, float) or "-" not in str(data):  # Check for NaN or no "-"
        lows.append(np.nan)
        highs.append(np.nan)
    else:
        
        try:
            low, high = data.split("-")
            low, high = float(low.strip()), float(high.strip())  # Strip spaces and convert
            lows.append(low)
            highs.append(high)
        except ValueError as e:  # Catch any unexpected errors
            print(f"Error processing '{data}': {e}")
            lows.append(np.nan)
            highs.append(np.nan)

HealthData['LowQR'] = lows
HealthData['HighQR'] = highs   

In [ ]:
HealthData.drop(['Quartile Range'], axis=1, inplace=True)

Data collection and manipulation

In [ ]:
# Define categorical and numerical features

numericalFeatures = ['Value','LowCI','HighCI','LowQR','HighQR']
others=['Group','State','Phase','Time Period','Time Period Start Date','Time Period End Date','Indicator']

# Create a preprocessor to handle missing values
preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='mean'), numericalFeatures),
        
    ]
)

# Fit and transform the data
transformedData = preprocessor.fit_transform(HealthData)
studentDataCleaned = pd.DataFrame(data= transformedData, columns= numericalFeatures)
otherdf= HealthData[others].copy()
HealthDataCleaned = pd.concat([studentDataCleaned,otherdf], axis=1)
HealthDataCleaned.head()

In [ ]:
HealthDataCleaned.shape

Add code to db

In [ ]:
# Checking for outliers.
fig, ax = plt.subplots(5, figsize=(20, 15))
ax = ax.flatten()
for i in range(len(ax)):
    x = HealthDataCleaned[numericalFeatures[i]]
    ax[i].boxplot(x)
    ax[i].set_title(numericalFeatures[i])
    ax[i].grid(visible=True)
    
plt.show()

Removing of outliers


In [ ]:
HealthDataCleaned = HealthDataCleaned[(HealthDataCleaned['Value'] < 59) & (HealthDataCleaned['LowCI'] < 59)]

In [ ]:
# Boxplot after removing outliers
fig, ax = plt.subplots(5, figsize=(20, 15))
ax = ax.flatten()
for i in range(len(ax)):
    x = HealthDataCleaned[numericalFeatures[i]]
    ax[i].boxplot(x)
    ax[i].set_title(numericalFeatures[i])
    ax[i].grid(visible=True)
    
plt.show()

Data visualisation

In [ ]:
# Histogram of all numeric features
fig, ax = plt.subplots(5, figsize=(15, 20))
ax = ax.flatten()
for i in range(len(ax)):
    x = HealthDataCleaned[numericalFeatures[i]]
    ax[i].hist(x, bins=5)
    ax[i].set_title(numericalFeatures[i])
plt.show()

In [ ]:
# Single dimension with discrete categories.
gender_counts = HealthDataCleaned['Phase'].value_counts()

# Plot a pie chart
plt.figure(figsize=(4, 4))
gender_counts.plot(kind='pie', autopct='%1.1f%%', startangle=140, colors=['lightseagreen', 'slateblue', 'lightgray'])
plt.title('Distribution of Phase')
plt.ylabel('')  # Remove the default y-axis label for better appearance
plt.show()

In [ ]:
# correlation matrix of all numeric features
plt.figure(figsize=(10, 8))
sns.heatmap(numerical_corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("numerical correlation")
plt.show()

Do barchart

In [ ]:
#Scatter plot : Relationship between observations on two continuous dimensions
# Scatter plot for Value vs HighCI
plt.figure(figsize=(10, 6))
plt.scatter(HealthDataCleaned['Value'], HealthDataCleaned['HighCI'])
plt.xlabel('Value')
plt.ylabel('HighCI')
plt.title('Scatter Plot of Value vs HighCI')
plt.show()

In [ ]:
HealthDataCleaned.info()

In [93]:

# Convert to datetime
HealthDataCleaned['Time Period Start Date'] = pd.to_datetime(HealthDataCleaned['Time Period Start Date'])
HealthDataCleaned['Time Period End Date'] = pd.to_datetime(HealthDataCleaned['Time Period End Date'])

# Convert to custom numeric format YYYYMMDD as float
HealthDataCleaned['Start Date Float'] = HealthDataCleaned['Time Period Start Date'].dt.strftime('%Y%m%d').astype(float)
HealthDataCleaned['End Date Float'] = HealthDataCleaned['Time Period End Date'].dt.strftime('%Y%m%d').astype(float)

# Drop the original columns
HealthDataCleaned = HealthDataCleaned.drop(columns=['Time Period Start Date', 'Time Period End Date'])

In [94]:
HealthDataCleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10404 entries, 0 to 10403
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Value             10404 non-null  float64
 1   LowCI             10404 non-null  float64
 2   HighCI            10404 non-null  float64
 3   LowQR             10404 non-null  float64
 4   HighQR            10404 non-null  float64
 5   Group             10404 non-null  object 
 6   State             10404 non-null  object 
 7   Phase             10404 non-null  object 
 8   Time Period       10404 non-null  int64  
 9   Indicator         10404 non-null  object 
 10  Start Date Float  10404 non-null  float64
 11  End Date Float    10404 non-null  float64
dtypes: float64(7), int64(1), object(4)
memory usage: 975.5+ KB


In [ ]:
HealthDataCleaned

In [96]:
categoricalFeatures=['Group','State','Phase']
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)
encoded_data = encoder.fit_transform(HealthDataCleaned[categoricalFeatures])
encoded_df = pd.DataFrame()
for i, col in enumerate(encoder.get_feature_names_out()):
    encoded_df[col] = encoded_data[:, i]
encoded_df

,Group_By Age,Group_By Disability status,Group_By Education,Group_By Gender identity,Group_By Presence of Symptoms of Anxiety/Depression,Group_By Race/Hispanic ethnicity,Group_By Sex,Group_By Sexual orientation,Group_By State,Group_National Estimate,...,State_Wisconsin,State_Wyoming,Phase_-1,Phase_2,Phase_3 (Jan 6 – Mar 29),Phase_3 (Oct 28 – Dec 21),Phase_3.1,Phase_3.2,Phase_3.3,Phase_3.4
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10399,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10400,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10401,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10402,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [101]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
HealthDataCleaned['Indicator'] = le.fit_transform(HealthDataCleaned['Indicator'])
